In [ ]:
#Kütüphanelerin eklenmesi
import numpy as np #Bu kütüphane lineer cebir için kullandığımız kütüphane fonksiyonlarını içeriyor
import pandas as pd # verilerimizi işlemek için pandas kütüphanasini kullanıyoruz(örn pd.read_scv)
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, cross_val_predict, cross_val_score
!pip install utils
from utils import *
import random
from spacy.tokenizer import Tokenizer
import warnings
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
import re
warnings.filterwarnings('ignore') 
import string
from sklearn.pipeline import make_pipeline
import xlsxwriter 
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import time
from spacy.lang.tr import Turkish
from sklearn.metrics import classification_report, accuracy_score, make_scorer,precision_score,recall_score,f1_score
from transformers import AutoModel, AutoTokenizer
from glove import Corpus, Glove
import fasttext
import fasttext.util
import nltk 
import torch

In [ ]:
column = ['tweets','duygu','preprocessing']
df = pd.read_excel("../dataset/total.xlsx")
df.columns=column
#veri setinin gösterilmesi
df=df.drop_duplicates()
df['tweets']=df['tweets'].str.lower()


In [ ]:
len(df[df.duygu==1]),len(df[df.duygu==0])

In [ ]:
df=df.sample(frac=1).reset_index(drop=True)

In [ ]:
# df.loc[df['duygu'] == "olumlu", "duygu"] = 1
# df.loc[df['duygu'] == "olumsuz", "duygu"] = 0

In [ ]:
def classification_report_with_accuracy_score(y_true, y_pred):
    print(classification_report(y_true, y_pred)) # print classification report
    
    return accuracy_score(y_true, y_pred) # return accuracy score

In [ ]:
def classification_report_with_precision_score(y_true, y_pred):
    
    return precision_score(y_true, y_pred, average='weighted')

In [ ]:
def classification_report_with_recall_score(y_true, y_pred):
    
    return recall_score(y_true, y_pred,average='weighted')

In [ ]:
def classification_report_with_f1_score(y_true, y_pred):
    
    return f1_score(y_true, y_pred,average='weighted')

In [ ]:
X_t = [word_tokenize(i) for i in df.tweets.tolist()]

In [ ]:
start=time.time()
n_features=32
window_size=5
min_count=10
epoch=50
n_workers=8

word2vec = Word2Vec(size = n_features,
            window = window_size, 
            min_count= min_count,
            workers = n_workers, 
            sg=0)
word2vec.build_vocab(X_t)
word2vec.train(X_t, 
            total_examples=word2vec.corpus_count,  
            epochs = epoch)
end=time.time()
print(end-start)

In [ ]:
nlp= Turkish()
def feature_extraction_word2vec(text):
    yeniMetin = ""
    for i in text:
        if i not in string.punctuation:
            yeniMetin += i
    vector=np.zeros(32) #400 uzunluğunda boş sıfırlarla bir vektör oluşturdum
    for token in nlp(yeniMetin):
        word = token.text.lower()
        if word not in ['',' ']:
            if word in word2vec.wv.vocab: #word2vec.wv.key_to_index.keys()
                vector+= word2vec.wv[word] #gensim modeliyle oluşturulan modeli kullandım
                #print(vector)
        
    return vector

In [ ]:
y_total= []
y_total=df.duygu.tolist()

X_total = []
for element in tqdm(df.tweets):
    X_total.append(feature_extraction_word2vec(element))
    

In [ ]:
# Nested CV with parameter optimization
clf = svm.SVC(kernel='linear')

accuracy = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_accuracy_score))
print("Accurancy for word2vec/SVM: ",accuracy.mean())


precision = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_precision_score))
print("Precision for word2vec/SVM: ",precision.mean())


recall = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_recall_score))
print("Recall for word2vec/SVM: ",recall.mean())


f1 = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_f1_score))
print("F1-Score for word2vec/SVM: ",f1.mean())

In [ ]:
# Nested CV with parameter optimization
clf = LogisticRegression()

accuracy = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_accuracy_score))
print("Accurancy for word2vec/LogisticRegression: ",accuracy.mean())


precision = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_precision_score))
print("Precision for word2vec/LogisticRegression: ",precision.mean())


recall = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_recall_score))
print("Recall for word2vec/LogisticRegression: ",recall.mean())


f1 = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_f1_score))
print("F1-Score for word2vec/LogisticRegression: ",f1.mean())


In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(10, 3), random_state=1)

accuracy = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_accuracy_score))
print("Accurancy for word2vec/MLPC: ",accuracy.mean())


precision = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_precision_score))
print("Precision for word2vec/MLPC: ",precision.mean())


recall = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_recall_score))
print("Recall for word2vec/MLPC: ",recall.mean())


f1 = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_f1_score))
print("F1-Score for word2vecF/MLPC: ",f1.mean())
